<a href="https://colab.research.google.com/github/aiswarya-1422/AI-SMART-SEARCH-for-Kerala-GOVT-/blob/main/prepro_fst_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-mal
!pip install pdfplumber pdf2image pytesseract googletrans==4.0.0-rc1 reportlab pillow



Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,628 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,305 kB]
Get:14 http://archive.u

In [ ]:
!unzip datasets.zip

Archive:  datasets.zip
   creating: datasets/
   creating: datasets/Circulars/
   creating: datasets/Circulars/English/
  inflating: datasets/Circulars/English/11-2019-FinDated05-02-2019_61.pdf  
  inflating: datasets/Circulars/English/4-2019-FinDated14-01-2019_61.pdf  
  inflating: datasets/Circulars/English/49 gov ord.compressed.pdf  
  inflating: datasets/Circulars/English/95-2018-FinDated16-10-2018_61.pdf  
  inflating: datasets/Circulars/English/CIR.09-11-Fin.pdf  
  inflating: datasets/Circulars/English/CIR.13-03-Fin.pdf  
  inflating: datasets/Circulars/English/CIR.15-02-Fin.pdf  
  inflating: datasets/Circulars/English/CIR.2-2010-FIN.pdf  
  inflating: datasets/Circulars/English/Cir.39-06-MLA.pdf  
  inflating: datasets/Circulars/English/CIR.56-2009-Fin.pdf  
  inflating: datasets/Circulars/English/CIR.59-10-Fin.pdf  
  inflating: datasets/Circulars/English/CIR.74-09-Fin.pdf  
  inflating: datasets/Circulars/English/CIR.77-07-76.pdf  
  inflating: datasets/Circulars/English/CIR

In [ ]:

import os
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
from googletrans import Translator
import json


INPUT_DIR = "/content/datasets"
OUTPUT_DIR = "/content/processed_json"
os.makedirs(OUTPUT_DIR, exist_ok=True)

translator = Translator()

def extract_text_from_pdf(pdf_path, lang="eng"):
    """Extract text from PDF. If no text layer, use OCR."""
    text = ""

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except:
        pass

    if not text.strip():
        print(f"[OCR] Processing scanned PDF: {pdf_path}")
        pages = convert_from_path(pdf_path, dpi=300, poppler_path='/usr/bin/')
        for page in pages:
            text += pytesseract.image_to_string(page, lang=lang) + "\n"

    return text.strip()

def translate_if_needed(text, src_lang, dest_lang="en"):
    """Translate Malayalam text to English; skip if already English."""
    if src_lang == "mal":
        translated_chunks = []
        chunk_size = 4000
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i+chunk_size]
            translated = translator.translate(chunk, src="ml", dest=dest_lang)
            translated_chunks.append(translated.text)
        return "\n".join(translated_chunks)
    else:
        return text

def process_pdf(pdf_path, doc_type, lang_folder):
    lang = "mal" if lang_folder.lower() == "malayalam" else "eng"
    extracted_text = extract_text_from_pdf(pdf_path, lang=lang)
    translated_text = translate_if_needed(extracted_text, lang)

    return {
        "file_name": os.path.basename(pdf_path),
        "doc_type": doc_type,
        "language": lang_folder,
        "original_text": extracted_text,
        "translated_text": translated_text
    }

def main():
    doc_types = ["Circulars", "GO"]
    lang_folders = ["English", "Malayalam"]

    for doc_type in doc_types:
        for lang_folder in lang_folders:
            folder_path = os.path.join(INPUT_DIR, doc_type, lang_folder)
            if not os.path.exists(folder_path):
                continue

            for file in os.listdir(folder_path):
                if file.endswith(".pdf"):
                    pdf_path = os.path.join(folder_path, file)
                    print(f"Processing: {pdf_path} ...")
                    data = process_pdf(pdf_path, doc_type, lang_folder)

                    json_filename = os.path.splitext(file)[0] + ".json"
                    output_path = os.path.join(OUTPUT_DIR, json_filename)
                    with open(output_path, "w", encoding="utf-8") as f:
                        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Processing complete. JSON files saved in {OUTPUT_DIR}/")

if __name__ == "__main__":
    main()

Processing: /content/datasets/Circulars/English/CircularNo39-2021-FinDated29-04-2021_61.pdf ...
Processing: /content/datasets/Circulars/English/Circular No 19-2014-Fin  dated 25-02-2014.pdf ...
Processing: /content/datasets/Circulars/English/CIR.No.67-08-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.No.67-08-Fin.pdf
Processing: /content/datasets/Circulars/English/CIR.95-09-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.95-09-Fin.pdf
Processing: /content/datasets/Circulars/English/11-2019-FinDated05-02-2019_61.pdf ...
Processing: /content/datasets/Circulars/English/CIR.No.82-08-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.No.82-08-Fin.pdf
Processing: /content/datasets/Circulars/English/CircularNo06-2022-FinDated05-02-2022_61.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CircularNo06-2022-FinDated05-02-2022_61.pdf
Processing: /content/datasets/Circulars/

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (151467696 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Processing: /content/datasets/Circulars/English/CircularNoITSF-02-07-2020-FinDated25-04-2020_61.pdf ...
Processing: /content/datasets/Circulars/English/49 gov ord.compressed.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/49 gov ord.compressed.pdf
Processing: /content/datasets/Circulars/English/CircularNo18-2021-FinDated26-02-2021_61.pdf ...
Processing: /content/datasets/Circulars/English/CIR.2-2010-FIN.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.2-2010-FIN.pdf
Processing: /content/datasets/Circulars/English/CIR.No.66-08-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.No.66-08-Fin.pdf
Processing: /content/datasets/Circulars/English/CircularNo31-2021-FinDated29-03-2021_61.pdf ...
Processing: /content/datasets/Circulars/English/Circular No 05-2018-Fin dated 09-01-2018.pdf ...
Processing: /content/datasets/Circulars/English/Circular No 72-2011-(18)-Fin  Dated  22-10-2011.pdf ...
[OCR] Processing 

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (146231875 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Processing: /content/datasets/Circulars/English/CircularNo77-2022-FinDated01-09-2022_61.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CircularNo77-2022-FinDated01-09-2022_61.pdf
Processing: /content/datasets/Circulars/English/Circular No 24-2013-Fin Dated 28-02-2013.pdf ...
Processing: /content/datasets/Circulars/English/CIR.09-11-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.09-11-Fin.pdf
Processing: /content/datasets/Circulars/English/Circular No 97-2017 -Fin dated 27-12-2017..pdf ...
Processing: /content/datasets/Circulars/English/CIR.56-2009-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/CIR.56-2009-Fin.pdf
Processing: /content/datasets/Circulars/English/Circular No 42-2011-3-fin dated 15-07-2011.pdf ...
[OCR] Processing scanned PDF: /content/datasets/Circulars/English/Circular No 42-2011-3-fin dated 15-07-2011.pdf
Processing: /content/datasets/Circulars/English/Circular No 28-17-Fin date

...

Processing: /content/datasets/GO/English/GO(P)No 213-2015-Fin Dated 05-06-2015.pdf ...
Processing: /content/datasets/GO/English/GO(Rt) No 5237-2014-Fin  dated 05-07-2014.pdf ...
Processing: /content/datasets/GO/English/GO.P.632-10-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO.P.632-10-Fin.pdf
Processing: /content/datasets/GO/English/GO(P) No 257-2015-Fin  dated 26-06-2015.pdf ...
Processing: /content/datasets/GO/English/GO.P.78-10-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO.P.78-10-Fin.pdf
Processing: /content/datasets/GO/English/GO(P)No119-2022-FinDated30-09-2022_61.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO(P)No119-2022-FinDated30-09-2022_61.pdf
Processing: /content/datasets/GO/English/GO(P)No 157-2013-Fin Dated 06-04-2013.pdf ...
Processing: /content/datasets/GO/English/GO(RT) No 8912-2011-Fin  Dated 02-12-2011.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO(RT) No 8912-2011-Fi

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (155401250 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Processing: /content/datasets/GO/English/GO(Rt)No1477-2021-FinDated17-02-2021_61.pdf ...
Processing: /content/datasets/GO/English/G.O.P.595-07-MLA.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/G.O.P.595-07-MLA.pdf
Processing: /content/datasets/GO/English/GO(Rt) No5528-201120110715_0008.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO(Rt) No5528-201120110715_0008.pdf
Processing: /content/datasets/GO/English/GO.P.213-2009-FIN.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO.P.213-2009-FIN.pdf
Processing: /content/datasets/GO/English/NewDoc 41.pdf ...
Processing: /content/datasets/GO/English/GO(Rt)No693-2019-FinDated29-01-2019_99.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO(Rt)No693-2019-FinDated29-01-2019_99.pdf
Processing: /content/datasets/GO/English/G.O (Rt) No.248-08.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/G.O (Rt) No.248-08.pdf
Processing: /content/datasets/GO/English/G

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (155401250 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Processing: /content/datasets/GO/English/GO.P.456-09-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO.P.456-09-Fin.pdf
Processing: /content/datasets/GO/English/GO(Ms)No 34-2017-Fin dated 21-01-2017.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO(Ms)No 34-2017-Fin dated 21-01-2017.pdf
Processing: /content/datasets/GO/English/G.O.P.543-04-MLA.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/G.O.P.543-04-MLA.pdf
Processing: /content/datasets/GO/English/GO(P) No 409-2014-Fin  dated 23-09-2014.pdf ...
Processing: /content/datasets/GO/English/GO(P) No 317-13-Fin  dated 04-07-2013.pdf ...
Processing: /content/datasets/GO/English/GO(P)No104-2020-FinDated05-08-2020_99.pdf ...
Processing: /content/datasets/GO/English/GO.P.684-10-Fin.pdf ...
[OCR] Processing scanned PDF: /content/datasets/GO/English/GO.P.684-10-Fin.pdf
Processing: /content/datasets/GO/Malayalam/GO(P) No 505-2014-Fin  dated 15-11-2014.pdf ...
Processing: /content/dat

In [ ]:
!zip -r /content/processed_json.zip /content/processed_json

  adding: content/processed_json/ (stored 0%)
  adding: content/processed_json/GO(P)No93-2020-FinDated08-07-2020_94.json (deflated 75%)
  adding: content/processed_json/Circular No 85-2014-Fin  dated 24-09-2014.json (deflated 72%)
  adding: content/processed_json/CircularNo24-2023-FinDated09-03-2023_61.json (deflated 75%)
  adding: content/processed_json/GO.P.633-10-Fin.json (deflated 78%)
  adding: content/processed_json/CircularNo33-2020-FinDated10-06-2020_61.json (deflated 46%)
  adding: content/processed_json/Circular No 43-17-Fin dated 29-05-2017.json (deflated 80%)
  adding: content/processed_json/CircularNo88-2019-FinDated11-11-2019_61.json (deflated 46%)
  adding: content/processed_json/GO.P.320-10-Fin.json (deflated 78%)
  adding: content/processed_json/GO(Rt)No4936-2021-FinDated13-07-2021_99.json (deflated 67%)
  adding: content/processed_json/CircularNo06-2022-FinDated05-02-2022_61.json (deflated 76%)
  adding: content/processed_json/GO(Rt)No4675-2021-FinDated25-06-2021_61.j